In [22]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
import re
import torch
import torch.nn as nn
from datasets import load_dataset
from IMDBDataset import IMDBDataset
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer

In [23]:
# Загрузка датасета IMDB
dataset = load_dataset('imdb')
# Проверяем доступность GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Определяем модель
model = 'roberta-base'

In [24]:
print(f'Наш набор данных: {dataset}')

Наш набор данных: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [25]:
# Проверка размера набора данных. Всего (50_000 отзывов)
print(f'Размер тренировочной выборки: {len(dataset["train"])}')
print(f'Размер тестовой выборки: {len(dataset["test"])}')

Размер тренировочной выборки: 25000
Размер тестовой выборки: 25000


#### Как видно из вывода, в данных присутствует много ненужных элементов, таких как HTML-символы, знаки препинания и другие.

In [26]:
print(f'Первый отзыв: {dataset["train"]['text'][0]}')
print(f'Метка 1-го отзыва: {dataset["train"]['label'][0]}')

Первый отзыв: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and fa

#### Избавляемся от лишних символов и преобразуем все данные в нижний регистр.

In [27]:
# Очистка текста
def cleaning_data(examples):
    # Регулярное выражение для удаления всех символов, кроме букв и цифр
    pattern = r'[^a-zA-Zа-яА-Я0-9 ]'
    # Очистка текста в батче
    examples["text"] = [re.sub(pattern, '', text.lower()) for text in examples["text"]]
    return examples


cleaning_dataset = dataset.map(cleaning_data, batched=True)

#### После очистки данные стали более упорядоченными, и теперь с ними можно эффективно работать.

In [28]:
print(f'Первый отзыв после очистки: {cleaning_dataset["train"]['text'][0]}')

Первый отзыв после очистки: i rented i am curiousyellow from my video store because of all the controversy that surrounded it when it was first released in 1967 i also heard that at first it was seized by us customs if it ever tried to enter this country therefore being a fan of films considered controversial i really had to see this for myselfbr br the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such as the vietnam war and race issues in the united states in between asking politicians and ordinary denizens of stockholm about their opinions on politics she has sex with her drama teacher classmates and married menbr br what kills me about i am curiousyellow is that 40 years ago this was considered pornographic really the sex and nudity scenes are few and far between even t

### 📌 Применяем токенизатор к полю `"text"` в данных

**Токенизатор** — это инструмент, который преобразует текст в числовые представления, которые могут быть использованы нейронными сетями.

---

### Принцип работы токенизатора

#### 1 Разбиение текста на токены
Токенизатор делит текст на небольшие части — **токены**. Существуют два основных метода:

**📌 Простое разбиение по словам**

Каждое слово становится отдельным токеном:

```plaintext
["Привет", ",", "мир", "!"]
```

**📌 Подслово (Subword Tokenization)**

Токенизатор использует алгоритмы, такие как **BPE (Byte Pair Encoding)** или **SentencePiece**, чтобы разбивать слова на подслова. Это помогает обработке **неизвестных слов**.

```plaintext
["При", "вет", ",", "ми", "р", "!"]
```

---

#### 2 Преобразование токенов в ID
Каждый токен сопоставляется с **уникальным числовым идентификатором (ID)**.
Эти ID берутся из словаря токенизатора, созданного при предобучении модели.

| Токен | ID |
|-------|----|
| "Привет" | 101 |
| "," | 102 |
| "мир" | 103 |
| "!" | 104 |

📌 Итоговый список ID для `"Привет, мир!"`:
```plaintext
[101, 102, 103, 104]
```

---

#### 3 Добавление **специальных токенов**
Некоторые модели (например, **RoBERTa**) требуют **дополнительных токенов**:

- `[CLS]` — начало текста (используется в задачах классификации).
- `[SEP]` — разделяет части текста (например, в задачах «вопрос-ответ»).
- `[PAD]` — заполняет последовательности до одинаковой длины.

---

#### 4 `truncation=True`
Обрезает текст до **максимальной длины**, если он слишком длинный.

---

#### 5 `padding=True`
Добавляет **паддинг** (заполнение) до максимальной длины в батче, чтобы все входные данные имели одинаковую длину.

**Пример работы паддинга**:
Есть текст `"Привет, мир!"`.
Допустим, модель требует, чтобы длина входных данных была **7 токенов**.
Тогда токенизированный текст **дополняется нулями**:
```plaintext
[1, 1, 1, 1, 0, 0, 0]
```
Здесь **0** — это паддинг-токены, которые **не влияют на результат**, а просто выравнивают последовательности.

---

#### 6 `attention_mask`
**Attention mask** показывает, какие токены являются реальными, а какие — **паддингом**.

📌 Пример:
```plaintext
Токены:        [<s>, 101, 102, 103, </s>, 0, 0]
Attention Mask: [  1,   1,   1,   1,    1, 0, 0]
```
- **1** означает реальный токен.
- **0** означает паддинг.

Это помогает модели **игнорировать** лишние паддинг-токены при обучении.

In [29]:
# Предобученный токинизатор
tokenizer = RobertaTokenizer.from_pretrained(model)


# Токинизируем текст
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)


tokenized_dataset = cleaning_dataset.map(tokenize_function, batched=True)
# Преобразуем данные в тензор для PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [30]:
# Датасет после токинизации
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [31]:
# Проверим данные
print(f'После преобразования input_ids: {tokenized_dataset['train']['input_ids']}')
print(f'После преобразования attention_mask: {tokenized_dataset['train']['attention_mask']}')
print(f'После преобразования label: {tokenized_dataset['train']['label']}')

После преобразования input_ids: tensor([[    0,   118, 16425,  ...,     1,     1,     1],
        [    0,   118,   524,  ...,     1,     1,     1],
        [    0,  1594,   129,  ...,     1,     1,     1],
        ...,
        [    0,  9226,   822,  ...,     1,     1,     1],
        [    0,   627, 18848,  ...,    41,   758,     2],
        [    0,   627,   527,  ...,     1,     1,     1]])
После преобразования attention_mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
После преобразования label: tensor([0, 0, 0,  ..., 1, 1, 1])


In [32]:
# Разделение данных на тренировочную и тестовую
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

In [33]:
# Инициализация датасетов для Dataloader
train_dataset = IMDBDataset(train_dataset)
test_dataset = IMDBDataset(test_dataset)

In [34]:
# Инициализация предобученной модели BERT для классификации последовательностей
# num_labels - указываем количество классов для классификации.
model = RobertaForSequenceClassification.from_pretrained(model, num_labels=2)
# Перевод модели на GPU
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [35]:
# Оптимизатор и функция потерь
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [36]:
# Подаём данные в DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, pin_memory=False)


```python
torch.cuda.empty_cache()
```
🔹 **Очищает неиспользуемую память GPU**, освобождая её от ненужных тензоров. Это помогает **избежать утечек памяти**.

---

```python
epochs = 3
for epoch in range(epochs):
```
🔹 Цикл по **количеству эпох** (проходов по всему датасету). Здесь у нас 3 эпохи.

---

```python
torch.cuda.empty_cache()
```
🔹 Очищаем кэш памяти GPU перед началом каждой эпохи, чтобы избежать избыточного использования памяти.

---

```python
model.train()
```
🔹 Переключаем модель в **режим обучения**. Это влияет на слои, такие как `Dropout` и `BatchNorm`, которые ведут себя по-разному в режиме `train()` и `eval()`.

---

```python
total_loss = 0
correct_predictions = 0
total_prediction = 0
```
🔹 **Обнуляем метрики** перед каждой эпохой:
- `total_loss` — накопленная функция потерь.
- `correct_predictions` — количество правильных предсказаний.
- `total_prediction` — общее количество примеров.

---

```python
for batch_idx, batch in enumerate(train_dataloader):
```
🔹 Проходим по **батчам** тренировочного датасета.

---

```python
optimizer.zero_grad()
```
🔹 **Обнуляем градиенты** перед вычислением нового градиента, чтобы избежать накопления градиентов от предыдущих шагов.

---

```python
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
labels = batch['label'].to(device)
```
🔹 **Перемещаем данные на GPU** (если доступен):
- `input_ids` — индексы токенов.
- `attention_mask` — маска внимания.
- `labels` — истинные метки классов.

---

```python
outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
logits = outputs.logits
```
🔹 **Прямой проход** через модель:
- `outputs.loss` — вычисленная функция потерь.
- `outputs.logits` — предсказанные логиты (до softmax).

---

```python
for param in model.parameters():
    if param.grad is not None and torch.isnan(param.grad).any():
        print("NaN в градиентах!")
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
```
🔹 **Проверка на NaN в градиентах** (если градиент стал `NaN`, обрезаем его).
⚠️ Это помогает избежать проблем с нестабильностью обучения.

---

```python
loss.backward()
```
🔹 **Обратное распространение ошибки** — вычисление градиентов весов.

---

```python
clip_grad_norm_(model.parameters(), max_norm=1.0)
```
🔹 **Ограничение градиентов** (gradient clipping).
Это предотвращает **взрыв градиентов**, ограничивая их норму `max_norm=1.0`. Взрывом градиентов называется ситуация, когда градиенты становятся слишком большие во время обратного распространения ошибки.

---

```python
optimizer.step()
```
🔹 **Обновление параметров модели** с использованием оптимизатора.

---

```python
total_loss += loss.item()
_, predicted = torch.max(logits, dim=1)
correct_predictions += (predicted == labels).sum().item()
total_prediction += labels.size(0)
```
🔹 **Обновляем метрики**:
- `total_loss` накапливает функцию потерь.
- `predicted` получает предсказанные классы (`argmax` по логитам).`dim=1` означает, что мы ищем максимум по столбцам (по классам) для каждого примера.
- `correct_predictions` увеличивается, если предсказание совпадает с истинным `label`.
- `total_prediction` считает общее число примеров.

In [37]:
torch.cuda.empty_cache()

# Обучения модели
epochs = 3
for epoch in range(epochs):
    torch.cuda.empty_cache()
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_prediction = 0

    for batch_idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        for param in model.parameters():
            if param.grad is not None and torch.isnan(param.grad).any():
                print("NaN в градиентах!")
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        loss.backward()

        # Нормируем градиенты
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(logits, dim=1)
        correct_predictions += (predicted == labels).sum().item()
        total_prediction += labels.size(0)
    accuracy = correct_predictions / total_prediction
    print(f'Эпоха {epoch + 1}/{epochs}, Точность: {accuracy:.2f}, Потери: {total_loss / len(train_dataloader):.2f}')

Эпоха 1/3, Точность: 0.91, Потери: 0.25
Эпоха 2/3, Точность: 0.95, Потери: 0.17
Эпоха 3/3, Точность: 0.97, Потери: 0.12


In [38]:
# Сохранение модели
model_save_dir = 'models/roberta_base_finetuned'
model.save_pretrained(model_save_dir)
tokenizer.save_pretrained(model_save_dir)
print(f"Модель и токенизатор успешно сохранены в директории: {model_save_dir}")

Модель и токенизатор успешно сохранены в директории: models/roberta_base_finetuned


In [39]:
torch.cuda.empty_cache()
model_save_dir = 'models/roberta_base_finetuned'
# Загружаем модель
model = RobertaForSequenceClassification.from_pretrained(model_save_dir).to(device)
model.eval()
print("Модель успешно загружена.")

correct_predictions = 0
total_prediction = 0

# Оценка модели
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        _, predicted = torch.max(logits, dim=1)
        correct_predictions += (predicted == labels).sum().item()
        total_prediction += labels.size(0)
accuracy = correct_predictions / total_prediction
print(f'Точность на тестовой выборки: {accuracy:.2f}')

Модель успешно загружена.
Точность на тестовой выборки: 0.95
